In [2]:
import numpy as np # except for numpy

In [3]:
# sigmoid function
def sigmoid(x, deriv=False):
    if deriv == False:
        return 1 / (1 + np.exp(-x))
    return x * (1 - x)

define X (input data) and y (output data)

In [4]:
np.random.seed(1)
# abstract 1080 RGB 64x64 images
X = np.random.randint(256, size=(1080, 64, 64, 3))
# each one representing one of 5 abstract categories
y = np.random.randint(5, size=(1080, 1))

In [ ]:
def model(X, y):
    '''
    create weights
    GradientDescent:
        feedforward
        backprop
        update the weights
    '''
    np.random.seed(1)
    w1 = 2 * np.random.random((4, 4, 5, 5)) - 1
    w2 = 2 * np.random.random((2, 2, 5, 3)) - 1
    w3 = 2 * np.random.random((1, 2)) - 1
    
    # (Input) -> (Conv2D+ReLU -> MaxPool) -> (Conv2D+ReLU -> MaxPool) -> (flatten -> FC)
    
    Z1 = conv_forward(X, w1, stride=2, zero_padding=3)
    Z1 = max_pool_forward(Z1, f=4, stride=2)
    Z2 = conv_forward(Z1, w2, stride=2)
    Z2 = max_pool_forward(Z2)
    y_ = FlattenFC(Z2)
    
    Z1__error, Z2__error, y__error = backprop(Z1, Z2, y_)
    
    
    

In [19]:
def conv_forward(A_prev, W, stride=1, zero_padding=0):
    
    m, A_prev_H, A_prev_W, _ = A_prev.shape
    W_H, W_W, _, Z_C = W.shape
    
    Z_H = int((A_prev_H - W_H + 2*zero_padding) / stride) + 1
    Z_W = int((A_prev_W - W_W + 2*zero_padding) / stride) + 1
    
    Z = np.zeros([m, Z_H, Z_W, Z_C])    
    print(Z.shape)
    A_prev_pad = np.pad(A_prev, ((0, 0), (zero_padding, zero_padding), (zero_padding, zero_padding), (0, 0)), mode='constant', constant_values = 0) # (m, A_prev_H + 2 * pad, A_prev_W + 2 * pad, A_prev_C)
    
    for i in range(m): # number of samples
        for h in range(Z_H): # result's height
            for w in range(Z_W): # result's width
                for c in range(Z_C): # result's number of channels
                    Z[i, h, w, c] = (A_prev_pad[i, h * stride : h * stride + W_H, w * stride : w * stride + W_W] * W[:, :, :, c]).sum()
                    
    return Z * (Z > 0)
                    

In [17]:
Z1.shape

(10, 16, 16, 5)

In [21]:
np.random.seed(1)
X = np.random.randint(256, size=(10, 64, 64, 3))
w1 = 2 * np.random.random((4, 4, 3, 5)) - 1
w2 = 2 * np.random.random((2, 2, 5, 10)) - 1
Z1 = conv_forward(X, w1, stride=2, zero_padding=3)
Z1 = max_pool_forward(Z1, f=4, stride=2)
Z2 = conv_forward(Z1, w2, stride=2)
Z2 = max_pool_forward(Z2)
Z2.shape

(10, 34, 34, 5)
(10, 8, 8, 10)


(10, 7, 7, 10)

In [5]:
def conv_backward(dZ, A_prev, W, stride, zero_padding):
    
    m, A_prev_H, A_prev_W, A_prev_C = A_prev.shape
    W_H, W_W, _, _ = W.shape
    m, A_H, A_W, A_C = dZ.shape
    
    dA_prev = np.zeros([m, A_prev_H, A_prev_W, A_prev_C])
    dW = np.zeros([W_H, W_W, A_prev_C, A_C])
    A_prev_pad = np.pad(A_prev, ((0, 0), (zero_padding, zero_padding), (zero_padding, zero_padding), (0, 0)), mode='constant', constant_values = 0) # (m, A_prev_H + 2 * pad, A_prev_W + 2 * pad, A_prev_C)
    dA_prev_pad = np.zeros([m, A_prev_H + 2 * zero_padding, A_prev_W + 2 * zero_padding, A_prev_C])
    
    for i in range(m): # number of samples
        for h in range(A_H): # result's height
            for w in range(A_W): # result's width
                for c in range(A_C): # result's number of channels
                    dA_prev_pad[i, h * stride : h * stride + W_H, w * stride : w * stride + W_W] += (W[:, :, :, c] * dZ[i, h, w, c]).sum()
                    dW[:,:,:,c] += (A_prev_pad[i, h * stride : h * stride + f_H, w * stride : w * stride + f_W] * dZ[i, h, w, c]).sum()
        dA_prev[i] = dA_prev_pad[i, zero_padding:-zero_padding, zero_padding:-zero_padding]
        
    return dA_prev, dW

In [12]:
def max_pool_forward(A_prev, f=2, stride=1):
    
    m, A_prev_H, A_prev_W, Z_C = A_prev.shape
    
    Z_H = int((A_prev_H - f) / stride) + 1
    Z_W = int((A_prev_W - f) / stride) + 1
    
    Z = np.zeros([m, Z_H, Z_W, Z_C])    
    
    for i in range(m): # number of samples
        for h in range(Z_H): # result's height
            for w in range(Z_W): # result's width
                for c in range(Z_C): # result's number of channels
                    Z[i, h, w, c] = A_prev[i, h * stride : h * stride + f, w * stride : w * stride + f, c].max()
    
    return Z

In [13]:
def max_pool_backward(dA, A_prev, f, stride):
    
    m, A_prev_H, A_prev_W, A_prev_C = A_prev.shape
    m, A_H, A_W, A_C = A_prev.shape
    
    Z = np.zeros([m, A_H, A_W, A_C])    
    
    for i in range(m): # number of samples
        for h in range(A_H): # result's height
            for w in range(A_W): # result's width
                for c in range(A_C): # result's number of channels
                    mask = A_prev[i, h * stride : h * stride + f, w * stride : w * stride + f, c] == A_prev[i, h * stride : h * stride + f, w * stride : w * stride + f, c].max()
                    dA_prev[i, h * stride : h * stride + f, w * stride : w * stride + f, c] += A_prev[i, h * stride : h * stride + f, w * stride : w * stride + f, c] * mask
    
    return dA_prev

In [9]:
def FlattenFC_forward(A_prev, W):
    flatX = np.zeros((A_prev.shape[0],np.prod(A_prev.shape[1:])))
    for i in range(A_prev.shape[0]):
        A_prev_flat[i] = A_prev[i].flatten()
    Z = np.dot(A_prev_flat, W)
    return Z

In [10]:
def softmax(A_prev):
    #this function will calculate the probabilities of each
    #target class over all possible target classes. 
    maxes = np.amax(A_prev, axis=1)
    maxes = maxes.reshape(maxes.shape[0], 1)
    e = np.exp(A_prev - maxes)
    dist = e / np.sum(e, axis=1, keepdims=True)
    return dist.argmax(axis=-1)